1
---
 If the input clock source operates at 16 MHz, what is the TCNT1 value needed to generate a time delay of 0.5 millisecond in normal mode?



frist we find the time one clock cycle takes 

In [40]:
oneCycleTime = 1/16000000

6.25e-8

we then find how time in seconds

In [28]:
time = 0.5/1000

0.0005

we then divide to get the # of clock cycles 

In [41]:
time / oneCycleTime

8000.000000000001

we then subtract that from the max number in the counter witch is 2^16

In [43]:
2^16 - 8000

57536

we then need to find the hex number that this is so that we can load it into the TCNT1

In [ ]:
57536 / 16 / 16 / 16 # e

14.046875

In [ ]:
(57536 / 16 / 16 / 16 - 14) * 16 # 0

0.75

In [ ]:
(57536 / 16 / 16 / 16 - 14 - 0) * 16^2 # c

12.0

0xE0C0 is the answer 

2
---
 If the clock operates at 10 MHz, what is the OCR0 value needed to generate time delay of 4 microseconds in CTC mode without any prescaler.

we can make a method that dose this

In [62]:
# fn takes time in seconds and nuber of cycels in Hz 

function ocr0(t, Hz)
    return t/(1/Hz)-1
end


function int_to_hex(n::Int)
    return "0x" * string(n, base=16)
end

int_to_hex(Int(ocr0(4/10^6,10*10^6)))

"0x27"

In [63]:
ocr0(4/10^6,10*10^6)

39.0

the answer is 0x27

3
---
Write an assembly program for Timer1 to generate a square wave of 16000 Hz to output on PB3 using a clock frequency of 16 MHz.

 


we need to be on or off for half of the HZ 

In [60]:
16000/2

8000.0

In [76]:
int_to_hex((2^16)-8000)

"0xe0c0"

we cant put "0x1f40" in OCR0
so what we can do is with TCNT1




<pre>
.text
.global square 

square:
push r16
push r17

BEGIN:
    SBI PORTB,3 ;PB5 = 1
    RCALL DELAY
    CBI PORTB,3 ;PB5 = 0
    RCALL DELAY
    RJMP BEGIN



DELAY:
; Load 16-bit value (0xe0c0) into TCNT1
    LDI R16, LOW(0xe0c0) ; Load low byte of value into R16
    LDI R17, HIGH(0xe0c0) ; Load high byte of value into R17
    OUT TCNT1L, R16 ; Write low byte to TCNT1L
    OUT TCNT1H, R17 ; Write high byte to TCNT1H
    LDI r16, 0
    OUT TCCR1A,R16 ;WGM11:10=00
    LDI r16 ,0x01
    OUT TCCR1B,R16 ;WGM13:12=00,CS=CLK normal mode 
again:
    IN r16, TIFR1
    SBRS R16,TOV1
    rjump again
    LDI R20,  1 << TOV1 
    OUT TIFR1,R20 ;clear TOV1 flag
    LDI R17,0
    OUT TCCR1B,R17 ;stop timer
    OUT TCCR1A,R17 ;
ret 

pop r17
pop r16
ret

</pre>

4
---
What is the advantage of using falling/rising edges instead of high/low voltages to detect events?

We are detecting when the voltage changes rather than the state of the voltage if we need to trigger one interrupt this could be useful because, if the voltage is low then the interrupt will only happen once, otherwise if we need to continuously interrupt the system then the voltage would be better 

5
---
What is the signal the following code generates at PORTB pin 5? If it is a periodical signal, what is its period? The clock frequency is 16MHz. 

<pre>
  LDI  R16,0x20  

  SBI  DDRB,5  

  LDI  R17,0  

  OUT  PORTB,R17  ; turns off led 

  LDI  R20,0x00

  OUT  TCCR0A,R20

BEGIN:  LDI  R20,0x3B ; set counter to 59

  OUT  TCNT0,R20 ; write 59 into the counter

  LDI  R20,0x01

  OUT  TCCR0B,R20 ;trun on the counter 0 normal mode 

AGAIN:  IN  R20,TIFR0

  SBRS  R20,0

  RJMP  AGAIN      ; run for 59 - 256 = 195 clocks 

  LDI  R20,0x0  

  OUT  TCCR0B,R20

  LDI  R20,0x01

  OUT  TIFR0,R20

  EOR  R17,R16

  OUT  PORTB,R17

  RJMP   BEGIN  
  </pre>

<pre>
4 clock 
then 195 
then 8 clocks

So it alternates for about 207 clocks. 

On and off it is 414 clocks


so the speed is 16MHz /417
</pre> 

In [77]:
16000000/417

38369.30455635492

the answer is $38.36930455635492$ KHz